# Scheduler Local Test


## Setup


* `make start-scheduler` 
* `make start-envoy`
* `make start-agent`
* `make start-mlserver`
* `make start-rclone`

## No Auth Example

In [1]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [2]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000",
      "availableMemoryBytes": "1199500",
      "numLoadedModels": 1,
      "overCommitPercentage": 20
    }
  ],
  "expectedReplicas": -1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1
}


In [3]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 0.0.0.0:9000
> User-Agent: curl/7.47.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< content-length: 196
< content-type: application/json
< date: Sat, 02 Apr 2022 07:45:51 GMT
< server: envoy
< x-envoy-upstream-service-time: 1039
< seldon-route: iris_1
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris_1","model_version":"1","id":"41656083-ed34-49b3-b122-65be1d5a52bb","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [4]:
!curl -v http://0.0.0.0:9000/v2/health/live -H "seldon-model: iris"

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)













* Connection #0 to host 0.0.0.0 left intact


In [5]:
!curl -v http://0.0.0.0:9000/v2/models/iris -H "seldon-model: iris"

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)














* Connection #0 to host 0.0.0.0 left intact
{"name":"iris_1","versions":[],"platform":"","inputs":[],"outputs":[],"parameters":{"content_type":null}}

In [6]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
    -plaintext \
    -import-path ../../apis \
    -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:iris \
    0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [7]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-02T07:45:42.800774356Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-02T07:45:42.800774356Z"
      },
      "modelDefn": {
        "meta": {
          "name": "iris"
        },
        "modelSpec": {
          "uri": "gs://seldon-models/mlserver/iris",
          "requirements": [
            "sklearn"
          ],
          "memoryBytes": "500"
        },
        "deploymentSpec": {
          "replicas": 1
        }
      }
    }
  ]
}


In [8]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000",
      "availableMemoryBytes": "1199500",
      "numLoadedModels": 1,
      "overCommitPercentage": 20
    }
  ],
  "expectedReplicas": -1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1
}


In [9]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [10]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-04-02T07:46:13.974650008Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-04-02T07:46:13.974650008Z"
      },
      "modelDefn": {
        "meta": {
          "name": "iris"
        },
        "modelSpec": {
          "uri": "gs://seldon-models/mlserver/iris",
          "requirements": [
            "sklearn"
          ],
          "memoryBytes": "500"
        },
        "deploymentSpec": {
          "replicas": 1
        }
      }
    }
  ],
  "deleted": true
}


In [11]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000",
      "availableMemoryBytes": "1200000",
      "overCommitPercentage": 20
    }
  ],
  "expectedReplicas": -1,
  "availableReplicas": 1
}


In [12]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)








* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


## Inline RClone Config Example

Before running this 
  * install minio in a kind cluster with Ansible.
  * Update ip address for exposed minio ip address in example below
  
Create rclone.conf changing the ip address for minio as appropriate

```
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://172.18.255.1:9000
```
  
 Copy iris model to minio
 
 ```
 rclone --config ./rclone.conf copy mlrepo/iris s3://test
 ```

In [ ]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"},\
            modelSpec: {uri:"s3://models/iris",\
                        storageConfig: { \
                        storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://172.18.255.2:9000\"}}" \
     } \
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## Multiple Version Example

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris-multi-version",\
                           "artifactVersion":1,\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris-multi-version",\
                           "artifactVersion":2,\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

In [2]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-04-01T12:44:19.592467897Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-04-01T12:44:19.592467897Z"
      },
      "modelDefn": {
        "meta": {
          "name": "iris",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/mlserver/iris",
          "requirements": [
            "sklearn"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}


In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'